# Masterclass definitiva: Regresión lineal y no lineal  
## Notebook paso a paso (teoría + ejemplos con números + gráficos + industria)

**Objetivo:** que el alumno entienda cada concepto con esta secuencia fija:  
**Definición real → Fórmula → Detalle (por qué) → Ejemplo con manzanitas → Ejemplo industrial → Importancia → Código + gráfico**

> Recomendación docente: ejecuta celda por celda y pregunta “¿qué significa este número?” antes de seguir.

---
## 0. Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

np.random.seed(42)

---
# PARTE 1 — REGRESIÓN LINEAL

## 1) Modelo lineal

### 1️⃣ Definición real  
Un **modelo lineal** predice un número (precio, ventas, salario, riesgo) como una **suma ponderada** de variables.

### 2️⃣ Fórmula  
\[
\hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_p x_p
\]

### 3️⃣ Detalle (qué significa cada término y por qué esa ecuación)
- \(\hat{y}\): predicción.  
- \(\beta_0\) (intercepto): valor “base” cuando todo \(x_j = 0\).  
- \(\beta_j\): **impacto marginal** (si \(x_j\) sube 1 unidad y lo demás se mantiene, \(\hat{y}\) cambia \(\beta_j\)).  
- Se usa esta forma porque es **interpretable**: cada variable “explica” una parte del resultado.

### 4️⃣ Ejemplo con manzanitas (números)  
Salario (S/) según años de experiencia:  
\(\hat{y} = 1200 + 500x\).  
Si \(x=4\) ⇒ \(\hat{y}=1200+2000=3200\).

### 5️⃣ Ejemplo industrial  
Riesgo crediticio aproximado:  
\(\widehat{Riesgo} = 0.02\cdot Deuda - 0.01\cdot Ingreso\).

### 6️⃣ Importancia  
- Es el estándar cuando necesitas **explicabilidad** (banca, salud, auditoría).  
- Es el punto de partida para casi todo lo demás (regularización, polinomios, etc.).

In [ ]:
# ✅ Código: ejemplo "salario vs experiencia" (fácil, con números entendibles)
X = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)              # experiencia (años)
y = np.array([1500, 2000, 2500, 3000, 3500])              # salario (S/.)

lin = LinearRegression()
lin.fit(X, y)

b0 = lin.intercept_
b1 = lin.coef_[0]

print("Modelo aprendido: y_hat = b0 + b1*x")
print("b0 (intercepto):", b0)
print("b1 (pendiente):", b1)

# Predicción paso a paso para x=4
x_new = np.array([[4]])
y_hat = lin.predict(x_new)[0]
print("\nPredicción para 4 años:", y_hat)

In [ ]:
# ✅ Gráfico: puntos + recta
y_pred = lin.predict(X)

plt.figure()
plt.scatter(X, y, alpha=0.8)
plt.plot(X, y_pred)
plt.xlabel("Experiencia (años)")
plt.ylabel("Salario (S/)")
plt.title("Regresión lineal: salario vs experiencia")
plt.show()

---
## 2) Función de costo (MSE): el “juez” del modelo

### 1️⃣ Definición real  
La **función de costo** mide *qué tan mal* está prediciendo el modelo.  
Entrenar = **minimizar** ese costo.

### 2️⃣ Fórmula (MSE / SSE normalizado)  
\[
J(\beta)=\frac{1}{2n}\sum_{i=1}^{n}(y_i-\hat{y}_i)^2
\]

### 3️⃣ Detalle (por qué esa ecuación)
- \(y_i - \hat{y}_i\): **residuo** (error) del punto \(i\).  
- Se eleva al cuadrado para:  
  1) evitar cancelación de signos,  
  2) castigar fuerte errores grandes,  
  3) hacer la función diferenciable y convexa (en regresión lineal).  
- El factor \(1/(2n)\) simplifica derivadas y normaliza por tamaño del dataset.

### 4️⃣ Ejemplo con manzanitas  
Si real = 210 y pred = 200 → error = 10 → error² = 100.  
Si error = 50 → error² = 2500 (mucho más castigo).

### 5️⃣ Ejemplo industrial  
En pricing: equivocarte por 1 sol en 1000 ventas es manejable; equivocarte por 50 soles en un segmento puede ser carísimo.  
MSE empuja al modelo a evitar errores grandes.

### 6️⃣ Importancia  
Sin función de costo, el modelo no tiene “meta” y no puede aprender.

In [ ]:
# ✅ Código: cálculo de errores y MSE sobre el ejemplo salario vs experiencia
residuos = y - y_pred
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)

print("Residuos (y - y_hat):", residuos)
print("MAE:", mae)
print("MSE:", mse)

In [ ]:
# ✅ Gráfico: residuos (qué tan lejos está cada punto de la recta)
plt.figure()
plt.scatter(X, residuos, alpha=0.8)
plt.axhline(0)
plt.xlabel("Experiencia (años)")
plt.ylabel("Residuo (y - y_hat)")
plt.title("Residuos: idealmente alrededor de 0")
plt.show()

---
## 3) OLS (Ordinary Least Squares)

### 1️⃣ Definición real  
**OLS** (Mínimos Cuadrados Ordinarios) es el método que encuentra los \(\beta\) que minimizan el MSE **sin penalización**.

### 2️⃣ Fórmula (solución cerrada)  
\[
\beta = (X^TX)^{-1}X^Ty
\]

### 3️⃣ Detalle (por qué importa)
- Es “solución directa” (si la matriz se puede invertir).  
- El problema: si hay **multicolinealidad** (variables muy correlacionadas), \(X^TX\) se vuelve inestable → coeficientes raros.

### 4️⃣ Ejemplo con manzanitas  
Si “edad” y “experiencia” son casi lo mismo, el modelo no sabe a cuál darle el efecto → coeficientes se vuelven inestables.

### 5️⃣ Ejemplo industrial  
En banca: ingreso, línea de crédito, gasto mensual suelen ir juntos (correlación alta). OLS puede “explotar”.

### 6️⃣ Importancia  
Es el baseline y el punto de comparación con Ridge/Lasso.

In [ ]:
# ✅ Código: demostrar multicolinealidad con un dataset sintético fácil
n = 200
x1 = np.random.normal(size=n)
x2 = x1 + np.random.normal(scale=0.05, size=n)  # casi igual a x1 (altamente correlacionada)
X_mc = np.column_stack([x1, x2])

# y depende realmente de x1
y_mc = 5*x1 + np.random.normal(scale=1.0, size=n)

ols_mc = LinearRegression()
ols_mc.fit(X_mc, y_mc)

print("Correlación entre x1 y x2:", np.corrcoef(x1, x2)[0,1])
print("Coeficientes OLS:", ols_mc.coef_)
print("Intercepto OLS:", ols_mc.intercept_)

In [ ]:
# ✅ Gráfico: correlación x1 vs x2 (visualiza multicolinealidad)
plt.figure()
plt.scatter(x1, x2, alpha=0.6)
plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Multicolinealidad: x2 ~ x1")
plt.show()

---
## 4) Regularización (concepto general)

### 1️⃣ Definición real  
**Regularizar** es añadir una “regla” al modelo para evitar que se vuelva demasiado sensible a los datos (overfitting) y para estabilizar coeficientes.

### 2️⃣ Fórmula general  
\[
J(\beta) = \underbrace{\frac{1}{2n}||y-X\beta||^2}_{\text{error}} + \lambda\cdot \underbrace{\Omega(\beta)}_{\text{penalización}}
\]

### 3️⃣ Detalle
- \(\Omega(\beta)\) define el tipo de regularización: L2 (Ridge), L1 (Lasso), etc.  
- \(\lambda\) controla “qué tan fuerte” castigas complejidad.  
  - \(\lambda=0\) → vuelves a OLS.  
  - \(\lambda\) grande → modelo más simple / más estable.

### 4️⃣ Ejemplo con manzanitas  
Es como decir: “Quiero predecir bien, pero **no te permito** usar coeficientes gigantes”.

### 5️⃣ Importancia industrial  
En producción, estabilidad > ajuste perfecto al entrenamiento.

---
## 5) Ridge Regression (L2)

### 1️⃣ Definición real  
**Ridge** es OLS + castigo por coeficientes grandes. Reduce varianza y estabiliza cuando hay multicolinealidad.

### 2️⃣ Fórmula  
\[
J(\beta)=\frac{1}{2n}||y-X\beta||^2 + \lambda||\beta||_2^2
\]

### 3️⃣ Detalle (por qué L2)
- L2 castiga “suavemente” valores grandes, encoge coeficientes.  
- No los hace cero, solo más pequeños.  
- Mantiene convexidad: mínimo global único.

### 4️⃣ Ejemplo con manzanitas  
Si OLS te da \(\beta=[10, -9]\) (muy extremos), Ridge puede llevarlos a \([6, -5]\).

### 5️⃣ Ejemplo industrial  
Modelos de riesgo con cientos de variables correlacionadas: Ridge suele ser más estable y robusto.

### 6️⃣ Importancia  
Cuando la prioridad es **estabilidad y generalización**.

In [ ]:
# ✅ Código: comparar OLS vs Ridge en el caso multicolineal (x1 ~ x2)
ridge_mc = Ridge(alpha=10.0)   # alpha = lambda (en sklearn)
ridge_mc.fit(X_mc, y_mc)

print("Coeficientes OLS :", ols_mc.coef_)
print("Coeficientes Ridge:", ridge_mc.coef_)

In [ ]:
# ✅ Gráfico: comparar coeficientes en barras (OLS vs Ridge)
labels = ["x1", "x2"]
ols_coef = ols_mc.coef_
ridge_coef = ridge_mc.coef_

xpos = np.arange(len(labels))

plt.figure()
plt.bar(xpos - 0.15, ols_coef, width=0.3, label="OLS")
plt.bar(xpos + 0.15, ridge_coef, width=0.3, label="Ridge")
plt.xticks(xpos, labels)
plt.title("Coeficientes: OLS vs Ridge (multicolinealidad)")
plt.legend()
plt.show()

---
## 6) Lasso Regression (L1)

### 1️⃣ Definición real  
**Lasso** es OLS + castigo L1. Además de estabilizar, puede hacer **selección de variables** porque empuja algunos coeficientes exactamente a cero.

### 2️⃣ Fórmula  
\[
J(\beta)=\frac{1}{2n}||y-X\beta||^2 + \lambda||\beta||_1
\]

### 3️⃣ Detalle (por qué L1)
- \(||\beta||_1 = \sum |\beta_j|\) genera geometría que favorece soluciones con ceros.  
- Resultado: modelo más simple y “feature selection” automático.

### 4️⃣ Ejemplo con manzanitas  
Tienes 5 variables, pero solo 2 importan. Lasso tiende a dejar coeficientes ≈ 0 para las inútiles.

### 5️⃣ Ejemplo industrial  
En marketing: cientos de variables (campañas, clicks, horarios, segmentos). Lasso ayuda a quedarte con las que realmente aportan.

### 6️⃣ Importancia  
Reduce complejidad, facilita interpretación y baja costo de producción (menos features).

In [ ]:
# ✅ Código: dataset donde solo 2 variables importan y el resto es ruido
n = 300
p = 8
X_fs = np.random.normal(size=(n, p))
true_beta = np.array([4.0, -3.0] + [0.0]*(p-2))     # solo x0 y x1 importan
y_fs = X_fs @ true_beta + np.random.normal(scale=1.0, size=n)

ols_fs = LinearRegression().fit(X_fs, y_fs)
ridge_fs = Ridge(alpha=10.0).fit(X_fs, y_fs)
lasso_fs = Lasso(alpha=0.1, max_iter=10000).fit(X_fs, y_fs)

print("True beta :", true_beta)
print("OLS beta  :", np.round(ols_fs.coef_, 3))
print("Ridge beta:", np.round(ridge_fs.coef_, 3))
print("Lasso beta:", np.round(lasso_fs.coef_, 3))

In [ ]:
# ✅ Gráfico: coeficientes (ver ceros en Lasso)
plt.figure()
plt.plot(true_beta, marker="o", label="True beta")
plt.plot(ols_fs.coef_, marker="o", label="OLS")
plt.plot(ridge_fs.coef_, marker="o", label="Ridge")
plt.plot(lasso_fs.coef_, marker="o", label="Lasso")
plt.title("Comparación de coeficientes: OLS vs Ridge vs Lasso")
plt.xlabel("Índice de variable")
plt.ylabel("Coeficiente")
plt.legend()
plt.show()

---
# PARTE 2 — REGRESIÓN NO LINEAL

## 7) Regresión polinomial

### 1️⃣ Definición real  
Se usa cuando la relación no es una recta. Se agregan términos como \(x^2, x^3\), etc., para capturar curvatura.

### 2️⃣ Fórmula  
\[
\hat{y}=\beta_0+\beta_1x+\beta_2x^2+\dots+\beta_nx^n
\]

### 3️⃣ Detalle (por qué se sigue llamando “lineal”)  
Porque es lineal en los coeficientes \(\beta\). El modelo sigue siendo una combinación lineal de parámetros.

### 4️⃣ Ejemplo con manzanitas  
Fertilizante vs cosecha:  
- poco fertilizante → baja cosecha  
- óptimo → máxima cosecha  
- demasiado → baja cosecha  
Esto es forma “U” (cuadrática).

### 5️⃣ Industria  
Economía (rendimientos decrecientes), ingeniería, fenómenos físicos.

### 6️⃣ Importancia  
Primer salto desde “recta” hacia “curvas” sin perder interpretabilidad.

In [ ]:
# ✅ Código + gráfico: datos con forma de U (cuadrática) y comparación lineal vs polinomial
X_u = np.linspace(-3, 3, 200).reshape(-1, 1)
y_u = (X_u[:,0]**2) + np.random.normal(scale=0.6, size=len(X_u))  # U + ruido

# Modelo lineal (falla)
lin_u = LinearRegression().fit(X_u, y_u)
y_lin_u = lin_u.predict(X_u)

# Modelo polinomial grado 2 (captura)
poly2 = PolynomialFeatures(degree=2, include_bias=False)
X_u_poly = poly2.fit_transform(X_u)      # [x, x^2]
poly_u = LinearRegression().fit(X_u_poly, y_u)
y_poly_u = poly_u.predict(X_u_poly)

plt.figure()
plt.scatter(X_u, y_u, alpha=0.4, label="Datos")
plt.plot(X_u, y_lin_u, label="Lineal")
plt.plot(X_u, y_poly_u, label="Polinomial grado 2")
plt.title("Lineal vs Polinomial (forma U)")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

print("R2 lineal:", r2_score(y_u, y_lin_u))
print("R2 poli2 :", r2_score(y_u, y_poly_u))

---
## 8) Modelos intrínsecamente no lineales (log / exp)

### 1️⃣ Definición real  
Son relaciones donde la forma funcional incluye logaritmos o exponenciales. Se usan cuando los incrementos tienen rendimientos decrecientes (log) o crecimiento acelerado (exp).

### 2️⃣ Fórmulas típicas
- Logarítmica: \(\hat{y}=\beta_0+\beta_1\ln(x)\)  
- Exponencial: \(\hat{y}=\beta_0 e^{\beta_1 x}\)

### 3️⃣ Detalle
- **Log**: los primeros aumentos importan mucho, luego “se aplana”.  
- **Exp**: crece rápido (viralidad, intereses compuestos, epidemias).

### 4️⃣ Ejemplos con manzanitas
- Log: satisfacción vs salario (subir de 1000 a 2000 impacta más que de 9000 a 10000).  
- Exp: usuarios virales (cada usuario trae más usuarios).

### 5️⃣ Industria
Finanzas (interés compuesto), epidemiología, crecimiento de usuarios, decaimiento físico.

In [ ]:
# ✅ Código + gráfico: ejemplo logarítmico (rendimientos decrecientes)
x = np.linspace(1, 100, 200)
y_log = 10 + 5*np.log(x) + np.random.normal(scale=0.5, size=len(x))

plt.figure()
plt.scatter(x, y_log, alpha=0.4)
plt.title("Ejemplo logarítmico: rendimientos decrecientes")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# ✅ Código + gráfico: ejemplo exponencial (crecimiento acelerado)
x = np.linspace(0, 3, 200)
y_exp = 2*np.exp(1.2*x) + np.random.normal(scale=0.8, size=len(x))

plt.figure()
plt.scatter(x, y_exp, alpha=0.4)
plt.title("Ejemplo exponencial: crecimiento acelerado")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

---
## 9) Árboles de decisión para regresión

### 1️⃣ Definición real  
Un **árbol de regresión** divide el espacio de variables en regiones (por reglas tipo “si x > 7”) y predice un valor constante (promedio) en cada región.

### 2️⃣ Lógica de splits  
El árbol elige cortes que reducen el MSE dentro de cada grupo.

### 3️⃣ Detalle
- No asume forma (ni recta ni curva).  
- Captura relaciones escalonadas o con umbrales.  
- Riesgo: sobreajuste si el árbol es muy profundo.

### 4️⃣ Ejemplo con manzanitas  
Precio de ticket de avión:
- ¿temporada alta?  
- ¿faltan menos de 7 días?  
- ¿fin de semana?  
La combinación te lleva a un “precio promedio” de ese segmento.

### 5️⃣ Industria  
Pricing, estimación de demanda, base de Random Forest / XGBoost / LightGBM.

In [ ]:
# ✅ Código + gráfico: árbol en datos con U (verás predicción escalonada)
tree = DecisionTreeRegressor(max_depth=3, random_state=42)
tree.fit(X_u, y_u)
y_tree = tree.predict(X_u)

plt.figure()
plt.scatter(X_u, y_u, alpha=0.4, label="Datos")
plt.plot(X_u, y_tree, label="Árbol (max_depth=3)")
plt.title("Árbol de regresión: predicción por tramos")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

print("R2 árbol:", r2_score(y_u, y_tree))

---
# PARTE 3 — EL MOTOR: OPTIMIZACIÓN

## 10) Gradiente descendente + tasa de aprendizaje

### 1️⃣ Definición real  
Es un algoritmo iterativo para minimizar una función. Se usa cuando no quieres/puedes resolver “de una” o cuando el dataset es enorme.

### 2️⃣ Fórmula  
\[
\beta_{t+1}=\beta_t-\eta \nabla J(\beta)
\]

### 3️⃣ Detalle
- \(\nabla J\): dirección de máxima subida (si restas, bajas).  
- \(\eta\): tamaño del paso (**learning rate**).  
  - muy pequeño → tarda mucho  
  - muy grande → rebota y puede divergir  
- En regresión lineal, el costo es convexo → no hay mínimos locales “trampa”.

### 4️⃣ Ejemplo con manzanitas  
Buscar el fondo de un tazón a oscuras: sientes la pendiente y das un paso hacia abajo.

In [ ]:
# ✅ Código: gradiente descendente en una función simple (beta-4)^2
def gradient_descent(beta0, eta, steps=20):
    beta = beta0
    history = []
    for t in range(steps):
        grad = 2*(beta - 4)      # derivada de (beta-4)^2
        beta = beta - eta*grad
        history.append(beta)
    return np.array(history)

hist_slow = gradient_descent(beta0=0, eta=0.05, steps=30)
hist_good = gradient_descent(beta0=0, eta=0.2,  steps=30)
hist_bad  = gradient_descent(beta0=0, eta=0.9,  steps=10)

print("Último beta (eta=0.05):", hist_slow[-1])
print("Último beta (eta=0.2): ", hist_good[-1])
print("Historial (eta=0.9):   ", hist_bad)

In [ ]:
# ✅ Gráfico: cómo cambia beta con diferentes learning rates
plt.figure()
plt.plot(hist_slow, marker="o", label="eta=0.05 (lento)")
plt.plot(hist_good, marker="o", label="eta=0.2 (bien)")
plt.title("Gradiente descendente: efecto del learning rate")
plt.xlabel("Iteración")
plt.ylabel("beta")
plt.legend()
plt.show()

---
# 11) Comparación final en un dataset “de juguete” (pero realista)

Aquí hacemos lo que suele pedir la industria:  
- separar train/test  
- entrenar varios modelos  
- comparar métricas  
- comparar interpretabilidad vs performance

In [ ]:
# ✅ Dataset sintético realista: y depende de 2 variables + ruido + variables irrelevantes
n = 800
p = 10
X_all = np.random.normal(size=(n, p))

beta_true = np.array([2.5, -1.7] + [0]*(p-2))
y_all = X_all @ beta_true + np.random.normal(scale=1.5, size=n)

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.25, random_state=42)

models = {
    "OLS": LinearRegression(),
    "Ridge(alpha=10)": Ridge(alpha=10.0),
    "Lasso(alpha=0.1)": Lasso(alpha=0.1, max_iter=10000)
}

rows = []
for name, m in models.items():
    m.fit(X_train, y_train)
    pred = m.predict(X_test)
    rows.append({
        "modelo": name,
        "MAE": mean_absolute_error(y_test, pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, pred)),
        "R2": r2_score(y_test, pred),
        "n_coef_~0": int(np.sum(np.isclose(getattr(m, "coef_", np.array([])), 0.0, atol=1e-6)))
    })

pd.DataFrame(rows).sort_values("RMSE")

In [ ]:
# ✅ Gráfico: coeficientes aprendidos (¿quién hace selección?)
coef_df = pd.DataFrame({
    "true": beta_true,
    "OLS": models["OLS"].coef_,
    "Ridge": models["Ridge(alpha=10)"].coef_,
    "Lasso": models["Lasso(alpha=0.1)"].coef_
})

plt.figure()
plt.plot(coef_df["true"].values, marker="o", label="true")
plt.plot(coef_df["OLS"].values, marker="o", label="OLS")
plt.plot(coef_df["Ridge"].values, marker="o", label="Ridge")
plt.plot(coef_df["Lasso"].values, marker="o", label="Lasso")
plt.title("Coeficientes: verdad vs modelos")
plt.xlabel("Índice de variable")
plt.ylabel("Coeficiente")
plt.legend()
plt.show()

coef_df

---
# 12) Guía de decisión (industrial)

**¿Qué usar y cuándo?**
- **OLS**: baseline, máxima interpretabilidad, pero sensible a multicolinealidad.  
- **Ridge**: muchas variables correlacionadas, quieres estabilidad.  
- **Lasso**: quieres reducir variables (feature selection), bajar complejidad.  
- **Polinomial**: relación curva (U/S) con sentido físico/económico.  
- **Árboles**: reglas/umbrales, datos con saltos, poca suposición de forma.  
- **Gradiente descendente**: dataset enorme / modelos más complejos; base de deep learning.

**Pregunta final para discusión:**  
> Si tienes 300 variables, muchas correlacionadas, y debes explicar el modelo al regulador: ¿Ridge o Árbol? ¿por qué?